In [26]:
from transformers import GPT2Model, GPT2Tokenizer, AutoTokenizer, GPT2Config, GPT2LMHeadModel, TextDataset,DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelWithLMHead, pipeline
from datasets import load_dataset
from transformers import TFAutoModelForSequenceClassification
from transformers import GPT2ForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [11]:
# load your dataset
df = pd.read_csv("gm_actions_preprocessed_more.csv")
df.head()

,short_description
0,automate td
1,review openapi spec diffs and docs in your prs
2,a neat summary of the changes you made to your...
3,test api compatibility of changes in your go l...
4,auto assign reviewer or assignee to pull reque...


# 1) Load the data (already preprocessed)



In [2]:
df = pd.read_csv("gm_actions.csv")
df.head()

,description
0,review openapi spec diffs and docs in your prs
1,a neat summary of the changes you made to your...
2,test api compatibility of changes in your go l...
3,auto assign reviewer or assignee to pull reque...
4,var various automated reviews is a deadly si...


# 2) Train Test Split

In [9]:
# using a 80:20 ratio, split X and Y into [x_train, x_test, y_train, y_test]
train,test = train_test_split(df, test_size=0.2, random_state=0)

# 3) Convert "test" and "train" into txt files

In [6]:
train.to_csv(r'train_ds.txt', header=None, index=None, sep=' ', mode='a')
test.to_csv(r'test_ds.txt', header=None, index=None, sep=' ', mode='a')

# 4) Call GPT tokenizer

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# set the train and set paths
train_path = r"C:\Users\Noor\Desktop\eecs 4080\Noor-Dennis\Noor\trained_model\train_ds.txt"
test_path = r"C:\Users\Noor\Desktop\eecs 4080\Noor-Dennis\Noor\trained_model\test_ds.txt"

# 6) Create func that takes the descriptions and puts them in a batch of dictionaries to be used by Pytorch

In [18]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

In [19]:
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

C:\Users\Noor\AppData\Roaming\Python\Python311\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (7932 > 1024). Running this sequence through the model will result in indexing errors


# 7) Load and train the model

In [23]:
model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-descriptions", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [24]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=48, training_loss=4.901604016621907, metrics={'train_runtime': 218.6108, 'train_samples_per_second': 0.837, 'train_steps_per_second': 0.22, 'total_flos': 11954110464000.0, 'train_loss': 4.901604016621907, 'epoch': 3.0})

# 8) Save the model

In [25]:
trainer.save_model()

# 9) Evaluate the model

In [41]:
prompt = pipeline('text-generation',model='gpt2-descriptions', tokenizer='gpt2')
result = prompt('this github action checks')[0]['generated_text']
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


this github action checks if an action can be used to change a value without the action getting deleted (see below). This is similar to the syntax for the actions in git subcommands except you get an array of options that your script can actually modify


<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(34,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(34,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.string, name=None))>


AttributeError: 'TrainingArguments' object has no attribute 'strategy'

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I really want my code to work but its not! I want to make it easy to use and maintain.\n\nI want to make it easy to use and maintain. I want to make it'}]


^C
